todo: move to lasagne recipes fork, and import the cifar10 data loader, and run exps on cuda4

In [1]:
import theano
from theano import tensor as T
import lasagne
from lasagne.layers import *
from lasagne.nonlinearities import *
from lasagne.objectives import *
from lasagne.regularization import *
from lasagne.random import get_rng
from lasagne.updates import *
from lasagne.init import *
import numpy as np
import sys
sys.path.append("../../modules/")
import helper as hp

from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams

import matplotlib.pyplot as plt
%matplotlib inline

import os
import cPickle as pickle

from theano.tensor import TensorType

from theano.ifelse import ifelse

from time import time

%load_ext rpy2.ipython

from scipy import stats

import deep_residual_learning_CIFAR10

import math

//anaconda/lib/python2.7/site-packages/Theano-0.8.0.dev0-py2.7.egg/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [40]:
class BinomialDropLayer(Layer):
    def __init__(self, incoming, nonlinearity=rectify, p=0.5,
                 **kwargs):
        super(BinomialDropLayer, self).__init__(incoming, **kwargs)
        self.nonlinearity = (identity if nonlinearity is None
                             else nonlinearity)
        self._srng = RandomStreams(get_rng().randint(1, 2147462579))
        self.p = p

    def get_output_for(self, input, deterministic=False, **kwargs):
        if deterministic:
            return self.p*input
        else:
            #mask = self._srng.binomial(n=1, p=(self.p), size=(input.shape[0],),
            #    dtype=input.dtype)
            mask = T.zeros((input.shape[0],)) + self._srng.uniform( (1,), 0, 1)[0]
            mask = mask.dimshuffle(0,'x','x','x')
            return mask*input

In [19]:
class IfElseDropLayer(Layer):
    def __init__(self, incoming, nonlinearity=rectify, p=0.5,
                 **kwargs):
        super(IfElseDropLayer, self).__init__(incoming, **kwargs)
        self.nonlinearity = (identity if nonlinearity is None
                             else nonlinearity)
        self._srng = RandomStreams(get_rng().randint(1, 2147462579))
        self.p = p

    def get_output_for(self, input, deterministic=False, **kwargs):
        if deterministic:
            return self.p*input
        else:
            return ifelse(
                T.lt(self._srng.uniform( (1,), 0, 1)[0], self.p),
                input,
                T.zeros(input.shape)
            )

There is a difference between this residual block method and the one that is defined in [link]. When the number of filters is different to the layer's output shape (or the stride is different), instead of using a convolution to make things compatible, we use an average pooling with a pool size of 1 and a the defined stride, followed by (if necessary) adding extra zero-padded feature maps. This is because this is how the authors in [link] have defined it.

In [59]:
def residual_block(layer, num_filters, filter_size=3, stride=1, num_layers=2):
    #print "input =", layer.output_shape
    conv = layer
    if (num_filters != layer.output_shape[1]) or (stride != 1):
        layer = Pool2DLayer(layer, pool_size=1, stride=stride, mode="average_inc_pad")
        diff = num_filters-layer.output_shape[1]
        if diff % 2 == 0: 
            width_tp = ((diff/2, diff/2),)
        else:
            width_tp = (((diff/2)+1, diff/2),)
        layer = pad(
            layer, 
            batch_ndim=1, 
            width=width_tp
        )
        #print "layer =", layer.output_shape
    for _ in range(num_layers):
        conv = Conv2DLayer(conv, num_filters, filter_size, stride=stride, pad='same')
        #print "conv =", conv.output_shape
        stride = 1
    nonlinearity = conv.nonlinearity
    conv.nonlinearity = lasagne.nonlinearities.identity
    return NonlinearityLayer(ElemwiseSumLayer([conv, layer]), nonlinearity)

layer = InputLayer( (None, 3, 32, 32) )
layer = Conv2DLayer(layer, 64, 7, stride=2, pad='same')
layer = Pool2DLayer(layer, 2)
for _ in range(3):
    layer = residual_block(layer, 64)
layer = residual_block(layer, 128, stride=2)
for _ in range(3):
    layer = residual_block(layer, 128)
layer = residual_block(layer, 256, stride=2)
for _ in range(5):
    layer = residual_block(layer, 256)
layer = residual_block(layer, 512, stride=2)
for _ in range(2):
    layer = residual_block(layer, 512)
layer = DenseLayer(layer, num_units=10)

In [60]:
l_out = layer

In [53]:
X = T.tensor4('X')
y = T.ivector('y')

net_out = get_output(l_out, X)
loss = categorical_crossentropy(net_out, y).mean()
params = get_all_params(l_out, trainable=True)
grads = T.grad(loss, params)
updates = nesterov_momentum(grads, params, learning_rate=0.01, momentum=0.9)
train_fn = theano.function(inputs=[X, y], outputs=loss, updates=updates)

In [54]:
l_out

In [55]:
fn_test = theano.function([X], get_output(l_out, X))

In [56]:
fn_test( np.ones((1, 3, 32, 32)) )

array([[ 0.08618267,  0.07042011,  0.05019732,  0.23297129,  0.32451911,
         0.        ,  0.        ,  0.        ,  0.        ,  0.05605585]])